In [2]:
# FIRST LINES OF THE Project CODE ?
import numpy as np
import pandas as pd

In [3]:
# This file is to import data fram ademe

import requests
import pandas as pd
import geopandas as gpd

In [8]:
# Recherche de la Liste des codes de communes

co_com = pd.read_csv("https://www.insee.fr/fr/statistiques/fichier/6051727/commune_2022.csv")
print(co_com.head())

# Sélection des communes de la région ile-de-France
CODES = co_com.loc[co_com["REG"]==11,"COM"].unique()
co_com2 = co_com.loc[co_com["REG"]==11,]
print(len(co_com2))
print(len(CODES))  #1288


  TYPECOM    COM   REG DEP CTCD  ARR  TNCC                    NCC  \
0     COM  01001  84.0  01  01D  012     5  ABERGEMENT CLEMENCIAT   
1     COM  01002  84.0  01  01D  011     5    ABERGEMENT DE VAREY   
2     COM  01004  84.0  01  01D  011     1      AMBERIEU EN BUGEY   
3     COM  01005  84.0  01  01D  012     1    AMBERIEUX EN DOMBES   
4     COM  01006  84.0  01  01D  011     1                AMBLEON   

                  NCCENR                  LIBELLE   CAN  COMPARENT  
0  Abergement-Clémenciat  L'Abergement-Clémenciat  0108        NaN  
1    Abergement-de-Varey    L'Abergement-de-Varey  0101        NaN  
2      Ambérieu-en-Bugey        Ambérieu-en-Bugey  0101        NaN  
3    Ambérieux-en-Dombes      Ambérieux-en-Dombes  0122        NaN  
4                Ambléon                  Ambléon  0104        NaN  
1288
1288


In [5]:
# Définition de la fonction d'extraction de données

def get_dpe_from_url(url):

    req = requests.get(url)
    wb = req.json()
    try:
        df = pd.json_normalize(wb["results"]) # Conversion du fichier Json en dataframe pandas
    except json.JSONDecodeError:
        print("A file was empty")
    
    if 'longitude' in df.columns.tolist() and 'latitude' in df.columns.tolist():
    # Conversion du dataframe classique en dataframe géographique 
        dpe = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["longitude"], df["latitude"]), crs = 4326) 
        dpe = dpe.dropna(subset = ['longitude', 'latitude'])

        return dpe


In [9]:
# Fixation des paramètres pour le requêtage
    #code_commune="01450"
code_commune = CODES[53]
size = 10000
root="https://data.ademe.fr/data-fair/api/v1/datasets/dpe-france/lines"
url_api = f"{root}?page=1&after=100&format=json&q_mode=simple&qs=code_insee_commune_actualise" + "%3A%22" + f"{code_commune}" + "%22" + f"&size={size}&select=" + "%2A&sampling=max"
url_api2 = f"{root}?after=100&format=json&q_mode=simple&qs=code_insee_commune_actualise" + "%3A%22" + f"{code_commune}" + "%22" + f"&size={size}&select=" + "%2A&sampling=neighbors"


In [6]:
# Fonction qui retourne l'url à partir du code_commune
def ret_url (cod,size_):
    return f"{root}?after=100&format=json&q_mode=simple&qs=code_insee_commune_actualise" + "%3A%22" + f"{cod}" + "%22" + f"&size={size_}&select=" + "%2A&sampling=neighbors"

In [ ]:
# Construction d'un dataframe complet des données pour la Région île-de-France

dpe_f = get_dpe_from_url(ret_url(CODES[0],10000))

for i in range(len(CODES)):                                                  # len(CODES)
    
    dpei = get_dpe_from_url(ret_url(CODES[i+1],10000))
    if dpei is not None:
        Commune = co_com2.loc[co_com2["COM"]==CODES[i],"NCC"].tolist()[0]
        dpei["Commune"] = Commune
        #dpei.set_index("Commune")
        dpe_f = pd.concat([dpe_f, dpei])
    
#dpe_f.head(5)
dpe_f.info()